In [2]:
# Install packages
%pip install networkx
%pip install numpy
%pip install tqdm
%pip install matplotlib
%pip install simanneal 
%pip install multiprocess
import sys# Start writing code here...


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -

In [1]:
from starter import *
from anneal import *
import math
import multiprocess as mp
import copy, random
from collections import defaultdict, Counter
import itertools
from heapq import heapify, heappop, heappush

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class GraphPartitioner(Annealer):
    class StateObject:
        def __init__(self, k):
            self.k = 0
            self.p = [0 for _ in range(k)]
            self.pi = [0 for _ in range(k)]
            self.norm_b = 0
            self.cost_p, self.cost_w, self.cost_k = 0, 0, 0
            self.total_cost = 0
            self.counter = 0
        def get_scores(self):
            return self.cost_w, self.cost_k, self.cost_p, self.total_cost
        
    def __init__(self, Graph: nx.Graph, k):
        
        self.G = Graph
        self.num_v = len(self.G.nodes)
        self.max_p = k
        self.state = self.StateObject(self.max_p)

        ### Below are all state variables
        self.k_way_random_part() # remember to add 1 to pi at the very end so no team is assigned 0
        # self.partition()
        
        b = [self.get_b_val(i) for i in range(len(self.state.p))]
        self.state.norm_b = np.linalg.norm(b)
        
        self.state.cost_p = math.exp(70 * self.state.norm_b)
        self.compute_cost_w_initial()
        self.compute_cost_k()
        self.state.total_cost = self.state.cost_k + self.state.cost_p + self.state.cost_w

        super(GraphPartitioner, self).__init__(self.state)  # important!
    
    def get_b_val(self, i):
        return self.state.p[i]/self.num_v - 1/self.state.k

    def partition(self):
        self.state.pi = [i % self.max_p for i in range(self.num_v)]
        # for i in range(self.num_v):
        #     self.state.pi[i] = i % self.max_p + 1
        
        self.state.k = len(set(self.state.pi))

        for v in range(self.num_v):
            self.state.p[self.state.pi[v]] += 1
    ### Sets up the teams in G and the b vector
    def k_way_random_part(self):
        self.state.pi = [i % self.max_p for i in range(self.num_v)]
        np.random.shuffle(self.state.pi)
        # self.state.pi = np.random.randint(self.max_p, size=len(self.num_v))
        self.state.k = len(set(self.state.pi))

        for v in range(self.num_v):
            self.state.p[self.state.pi[v]] += 1

    def compute_cost_k(self):
        self.state.cost_k = 100 * math.exp(self.state.k * 0.5)

    def compute_cost_p_on_swap(self, i, j):
        b_i, b_j = self.get_b_val(i), self.get_b_val(j)
        new_norm_b_squared = self.state.norm_b**2 - b_i**2 - b_j**2 + (b_i-1/self.num_v)**2 + (b_j+1/self.num_v)**2
        
        if math.isclose(new_norm_b_squared, 0, abs_tol=1e-9):
            new_norm_b_squared = 0
        self.state.norm_b = new_norm_b_squared ** 0.5
        self.state.cost_p = math.exp(70 * self.state.norm_b)
    
    def compute_cost_p_from_scratch(self):
        b = [i/self.num_v - 1/self.state.k for i in self.state.p if i != 0]
        self.state.norm_b = np.linalg.norm(b)
        self.state.cost_p = math.exp(70 * self.state.norm_b)
        
    def compute_cost_p(self, new):
        pass

    def compute_cost_w_initial(self):
        for u, v in self.G.edges:
            if self.state.pi[u] == self.state.pi[v]:
                self.state.cost_w += self.G[u][v]['weight']

    def compute_cost_w(self, u, i , j):
        if i == j:
            return
        for v in self.G[u]:
            if self.state.pi[v] == i:
                self.state.cost_w -= self.G[u][v]['weight']
            elif self.state.pi[v] == j:
                self.state.cost_w += self.G[u][v]['weight']
    
    def compute_total_cost(self, v, i, j):
        ### Compute new cost_k if needed
        k_changed = False
        if self.state.p[i] == 1: 
            self.state.k -= 1
            k_changed = True
        if self.state.p[j] == 0:
            self.state.k += 1
            k_changed = True
        if k_changed or self.state.counter % 100 == 0:
            self.compute_cost_k()
            self.state.p[i] -= 1
            self.state.p[j] += 1
            self.compute_cost_p_from_scratch()
        else:
            self.compute_cost_p_on_swap(i, j)
            self.state.p[i] -= 1
            self.state.p[j] += 1
            
        self.compute_cost_w(v, i, j)
        
        self.state.pi[v] = j

        self.state.total_cost = self.state.cost_k + self.state.cost_p + self.state.cost_w
        return self.state.total_cost

    def move_vertex(self):
        v = np.random.randint(self.num_v)
        i = self.state.pi[v]
        choices = [i for i in range(self.max_p)if self.state.p[i] != 0]
        if i in choices:
            choices.remove(i)
        j = choices[np.random.randint(self.state.k-1)]

        # j = 0 if i else 1
        self.compute_total_cost(v, i, j)
        
    def move_vertex_greedy(self, num_to_move):
        all_v = []
        heapify(all_v)

        vertex_weights = [[0 for j in range(self.max_p)] for _ in range(self.num_v)]
        
        for u, v in self.G.edges:
            pi_u, pi_v =  self.state.pi[u], self.state.pi[v]
            vertex_weights[u][pi_v] += self.G[u][v]['weight']
            vertex_weights[v][pi_u] += self.G[u][v]['weight']

        vert = np.arange(self.num_v)
        np.random.shuffle(vert)
        for v in vert:
            pi_v = self.state.pi[v]
            vertex_weights[v] = [vertex_weights[v][k] - vertex_weights[v][pi_v] for k in range(self.max_p)]
            v_max_diff, k = 10**9,0
            
            for j, w in enumerate(vertex_weights[v]):
                if w < v_max_diff:
                    k = j
                    v_max_diff = w
                    
            heappush(all_v, (v_max_diff, np.random.uniform(),v, k))
            
        # orig_ener = self.energy()
        # arr = []
        for i in range(num_to_move):
            diff,rand, v, k = heappop(all_v)
            # arr.append(diff)
            origin = self.state.pi[v]
            
            self.compute_total_cost(v, origin, k)
            # print('delta', self.energy()-orig_ener)
            # if self.energy()-orig_ener > 0:
            #     print(v, origin,k)
            #     print(vertex_weights[v])
            # orig_ener = self.energy()
        # print(arr)
        # print()
        
    def swap_two_vertex(self):

        i = j = 0
        while i == j:
            u, v = random.sample(range(self.num_v), 2)
            i,j = self.state.pi[u], self.state.pi[v]
        # print(u,v,i,j)
        # j = 0 if i else 1
        self.compute_total_cost(u, i, j)
        self.compute_total_cost(v, j, i)
  
            
    def add_one(self, reverse = False):
        for v in self.G.nodes:
            self.state.pi[v] += 1 * (-1) ** reverse
    
    def get_scores(self):
        return self.state.get_scores()

    def energy(self):
        return self.state.total_cost

    def move(self, num_moves = 5):
        move = self.state.counter % 2
        # move = 3
        if move == 0:
            self.move_vertex()
        elif move == 1:
            self.swap_two_vertex()
        else:
            self.move_vertex_greedy(num_moves)
        self.state.counter += 1

    def apply_pi(self):
        for v in range(self.num_v):
            self.G.nodes[v]['team'] = self.state.pi[v] + 1
            
    def update(self, step, T, E, acceptance, improvement):
        pass
            
def apply_pi(G, pi):
    num_v = len(G.nodes)
    assert num_v == len(pi)
    max_p = max(pi)

    p = list(Counter(pi))
    p.sort()
    map_p = {p[i]:i for i in range(len(p))}
    for v in range(num_v):
        G.nodes[v]['team'] = int(map_p[pi[v]] + 1)
    return G       

In [3]:
def run(solver, in_file: str, out_file: str, file_name, overwrite,df,target=280,target_k=None):
    instance = read_input(in_file)
    
    ret = solver(instance,in_file, out_file, file_name,target,target_k)
    if ret == None: return
    
    # cost, k, pi, state = ret
    
    # if not math.isclose(score(instance), cost):
    #     print(score(instance, separated=True))
    #     print(state.get_scores())
    #     return instance, pi, state
    #     # print(pi)
    #     # print([instance.nodes[v]['team'] for v in instance.nodes])
        
    # r = df.index[df['name'] == file_name]
    # overwrite = cost < float(df.iloc[r]['score']) - 10**-2
    # # print(cost,float(df.iloc[r]['score']) )
    # if overwrite:
    #     write(instance, in_file, out_file, file_name, cost, k, overwrite)
        
def save_and_plot(files, scores, output_name):
    counts, bins = np.histogram(scores)
    plt.stairs(counts, bins)
    
    scores_and_names = list(zip(files, scores))
    scores_and_names.sort()
    cat_size = len(scores)//3
    
    second = lambda x: x[1]
    large = sum(map(second, scores_and_names[:cat_size]))/cat_size
    medium = sum(map(second, scores_and_names[cat_size:cat_size*2]))/cat_size
    small =  sum(map(second, scores_and_names[cat_size*2:]))/cat_size
    
    os.makedirs('scores/' + output_name, exist_ok=True)  
    f = open('scores/' + output_name + '/avg_scores.txt', "w")
    f.write("small_avg: {0} \nmedium_avg: {1} \nlarge_avg: {2}".format(small, medium, large))
    f.close()
    
    df = pd.DataFrame(zip(files, scores), columns=['name', 'score'])
    df.to_csv('scores/' + output_name + '/out.csv')

def run_encapsulate(inp):
    target,target_k = 1, None
    if len(inp) == 6:
        file, solver, in_dir, out_dir,overwrite,df = inp
    else:
        file, solver, in_dir, out_dir,overwrite,df,target,target_k = inp    
    r = df.index[df['name'] == file]
    
    err = run(solver, str(Path(in_dir) / file), str(Path(out_dir) / f"{file[:-len('.in')]}.out"), file, overwrite,df,target,target_k)
    if err:
        return err[2]
    return
    
def run_all(solver, in_dir, out_dir, overwrite: bool=False):
    scores = defaultdict()
    files = [x for x in os.listdir(in_dir) if x.endswith('.in')]
    # files.sort(reverse=True)
    random.shuffle(files)
    for file in tqdm(files):
        ### only run if havent done so before
        r = df.index[df['name'] == file]
        if float(df.iloc[r]['score']) >= 200000:
            err = run(solver, str(Path(in_dir) / file), str(Path(out_dir) / f"{file[:-len('.in')]}.out"), file, overwrite)
            if err:
                return err[2]
        pass
    return None

def write(instance, in_file, out_file, file_name, cost, k, overwrite):
    df = pd.read_csv('best_scores.csv')
    r = df.index[df['name'] == file_name]
    original = float(df.loc[r, 'score'])
    df.loc[r, 'score'] = cost
    df.loc[r, 'k'] = k
    df.to_csv('best_scores.csv', index=False)
    print(f"{str(in_file)}: cost", original, '->',score(instance))
    write_output(instance, out_file, overwrite)

In [4]:
tar('outputs', overwrite = True)

In [5]:
def solve(G: nx.Graph,in_file, out_file, file_name, target=1,target_k=None):
    NUM_ITER_OUT,NUM_ITER_IN = 3,6
    df = pd.read_csv('best_scores.csv')
    r = df.index[df['name'] == file_name]
    min_cost = float(df.iloc[r]['score'])
    if min_cost <= target:
        return
    num_p = target_k if target_k else int(df.iloc[r]['k'])
    print(file_name, target, num_p)

    pi = []
    best_k = 0
    best_state= None
    epsilon = 10**-2
    num_v = len(G.nodes)
    size_to_time = {100:1.5, 300:2, 1000:2.5}
    time = size_to_time[num_v]
    found = False
    for k in range(num_p,num_p-1,-1):
        if k == 1:
            break
        for i in range(NUM_ITER_OUT):
            found =  min_cost <= target
            if found: break
            
            tourney = GraphPartitioner((G), k)
            tourney.set_schedule(tourney.auto(minutes=time,steps=1000))
            cost = tourney.energy()
            if cost < min_cost - epsilon:
                min_cost, pi, best_k = cost, tourney.state.pi.copy(), tourney.state.k
                apply_pi(G, pi)
                write(G, in_file, out_file, file_name, cost, k, True)
                print(tourney.state.get_scores())
            for _ in range(NUM_ITER_IN):
                # print(file_name, tourney.state.get_scores())
            
                found =  min_cost <= target
                if found: break
                partitioning, cost = tourney.anneal()

                if cost < min_cost - epsilon:
                    min_cost, pi, best_k = cost, tourney.state.pi.copy(), tourney.state.k
                    apply_pi(G, pi)
                    write(G, in_file, out_file, file_name, cost, k, True)
                    print(tourney.state.get_scores())
        if found:
            apply_pi(G, pi)
            return min_cost, best_k, pi, best_state
        print()

    return min_cost, best_k, pi, best_state

In [6]:
def get_targets(file):
    f = open(file, 'r')
    lines = f.readlines()
    targets = []
    for line in lines:
        name, num, k = line.strip().split()
        targets.append((name, float(num), int(k)))
    return targets
all_targets = get_targets('target_soln_new.txt')
len(all_targets)
# np.random.shuffle(targets)
files = [x for x in os.listdir('inputs') if x.endswith('.in')]

415

In [7]:
pool = mp.Pool(mp.cpu_count())
targets = all_targets
for _ in range(2):
    df = pd.read_csv('best_scores.csv')
    
    np.random.shuffle(targets)
    pool.map(run_encapsulate, [(file, solve, 'inputs', 'outputs',True, df,target,k) for file,target,k in tqdm(targets)])

# pool.map(run_encapsulate, [(file, solve, 'inputs', 'outputs',True, df) for file in tqdm(files)])

pool.close()

100%|██████████| 415/415 [00:00<00:00, 1513596.66it/s]

large20.inlarge241.inmedium150.insmall24.inmedium89.inmedium175.in    medium19.in 55377.32591025766 449.237726695157662750.72848132418large144.in 156489.02581064697 2009.563692318766717661.165042308323    132186.1042314803612 4701.43176316113 12143 
6 
10

14

7



inputs/medium175.in: cost 33534.55997845206 -> 32586.75320223392
(17742, 14841.31591025766, 3.437291976256717, 32586.753202233915)
inputs/medium89.in: cost 4141.553692318767 -> 4088.9279182525956
(2076, 2008.5536923187667, 4.374225933829153, 4088.927918252596)
inputs/medium89.in: cost 4088.927918252596 -> 2730.608892285611
(718, 2008.5536923187667, 4.055199966844682, 2730.6088922856115)
inputs/medium175.in: cost 32586.753202233915 -> 31368.303390619996
(16524, 14841.31591025766, 2.9874803623342947, 31368.303390619993)
inputs/medium175.in: cost 31368.303390619996 -> 29939.360125370233
(15091, 14841.31591025766, 7.044215112572992, 29939.360125370233)
inputs/large20.in: cost 68841.31591025766 -> 68412.00044834877
(28068, 40342.87934927351, 1.1210990752638181, 68412.00044834879)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=beec2354-b280-46f2-b093-65afd3bc9b88' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>